In [15]:
import os
import warnings
import datetime as dt
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly_express as px
import pmdarima as pm
import vaex as vx
from dateutil import parser
from matplotlib import dates as mpl_dates
from scipy.signal import savgol_filter
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller, arma_order_select_ic
import dtypes
# from tensorflow.keras.layers import LSTM, Dense, Flatten
# univariate lstm example
# from tensorflow.keras.models import Sequential

warnings.filterwarnings('ignore')

%matplotlib widget
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [16]:
plt.rcParams.update({
    "lines.color": "white",
    "patch.edgecolor": "white",
    "text.color": "black",
    "axes.facecolor": "white",
    "axes.edgecolor": "lightgray",
    "axes.labelcolor": "white",
    "xtick.color": "white",
    "ytick.color": "white",
    "grid.color": "lightgray",
    "figure.facecolor": "black",
    "figure.edgecolor": "black",
    "savefig.facecolor": "black",
    "savefig.edgecolor": "black"})

plt.style.use('dark_background')


In [17]:

# Imports data
path = os.path.dirname(os.getcwd())

def parse_dates(x):
    return dt.datetime.strptime(x, '%Y-%m-%d')


# You can try both to check speed of import
try:
    data = vx.open('%s//data//bigdata.hdf5'%path)
except (FileNotFoundError):
    data = vx.read_csv('%s//data//consumption_data.csv' % path, dtype=dtypes,
                       parse_dates=['Sale Date'], date_parser=parse_dates,
                       chunk_size=500_000, convert='%s//data//bigdata.hdf5' % path)


In [18]:
# Stripping Facility Names of Spaces and filtering only active facilities
data['Sale Facility'] = data['Sale Facility'].apply(lambda x: x.strip()).values

# Drop All Missing Values
data = data.dropmissing(['Product Name', 'Sale Facility'])


In [19]:
# Import VFL_CHECK
vfl_check = pd.read_csv('%s//data//vfl_check.csv' % path)
vfl_check['DISPENSATION VFL'] = vfl_check['DISPENSATION VFL'].apply(
    lambda x: x.strip())
vfl_list = vfl_check['DISPENSATION VFL'].tolist()
vfl_vx = vx.from_pandas(vfl_check)

# Import VDL Check
vdl_check = pd.read_csv('%s//data//vdl_check.csv' %
                        path, encoding='cp1252', dtype={"Product ID": "str"})
vdl_check.drop_duplicates(['Product Description'], inplace=True)
vdl_check['Product Description'] = vdl_check['Product Description'].apply(
    lambda x: x.strip())
vdl_check['Product ID'] = vdl_check['Product ID'].apply(
    lambda x: str(x.strip('GH-')))
vdl_check['Corresponding Quantity'] = vdl_check['Corresponding Quantity'].apply(
    lambda x: int(re.findall(r'\d+', x)[0]))
vdl_vx = vx.from_pandas(vdl_check)


In [20]:
def new_assign(vx_df, columns):
    for i in columns:
        vx_df[i] = vx_df[i].values.data


In [21]:
# Filtering all active facilities
filtered_data = data[data['Sale Facility'].isin(vfl_list)]
filtered_data = filtered_data.extract()


In [22]:
# Join the VDL Check Data with Filtered Data on Product IDs to get Categories
filtered_data.join(vdl_vx, how='left', left_on='Vdl Drug ID',
                   right_on='Product ID', inplace=True)


# Assigning the array data of the VDl Categories to the VDL Categories column since its a masked_array
new_assign(filtered_data, vdl_check.columns.tolist())

# Dropping unnecessary data after the join
filtered_data.drop(['Product ID'], inplace=True)

# Column for Packs Sold
filtered_data['Packs_Sold'] = (
    filtered_data['Sum of Quantity In Units'] / filtered_data['Corresponding Quantity']).values


In [23]:
# Joining vfl data wtih filtered_data to get facility locations
filtered_data.join(vfl_vx, how='left', left_on='Sale Facility',
                   right_on='DISPENSATION VFL', inplace=True)
# Dropping unnecessary data columns
filtered_data.drop(['SC VFL', 'DISPENSATION VFL'], inplace=True)


#,Sale Facility,Sale Date,Customer Type,Vdl Drug ID,Vdl Drug Display Name,Product Name,Item Category,Item Sub Category,Sum of Quantity In Units,Sum of Quantity In Packs,Product Description,Form,Strength,Brand / Proprietary Name,Manufacturer,OTC/POM,Tier,VDL Sub Category,Corresponding Quantity,Packs_Sold,LOCATION
0,A & F Pharmacy,2022-11-22 00:00:00.000000000,guest,107462,Para-Denk 125mg Suppositories x10,Para-Denk 125mg Suppositories x10,MEDICINES AND TREATMENTS,Pain,2,0.2,Para-Denk 125mg Suppositories x10,Suppository,125MG,PARA-DENK,Denk Pharma,OTC,Tier 2,Analgesics,10,0.2,GREATER ACCRA
1,A & F Pharmacy,2022-11-22 00:00:00.000000000,guest,108558,Naklofen Duo 75mg Tablets x20,Naklofen Duo 75mg Tablets x20,MEDICINES AND TREATMENTS,Pain,10,0.5,Naklofen Duo 75mg Tablets x20,Tablet,75MG,NAKOLFEN DUO,Krka,POM,Tier 2,Analgesics,20,0.5,GREATER ACCRA
2,A & F Pharmacy,2022-11-22 00:00:00.000000000,guest,126272,Tagera-Forte 1g (2 Doses) Tablets x1,Tagera-Forte 1g (2 Doses) Tablets x1,no data,no data,1,0.1,Tagera-Forte 1g (2 Doses) Tablets x1,Tablet,1g,TAGERA-FORTE,Unichem,POM,Tier 3,Anti-Infectives,1,1.0,GREATER ACCRA
3,A & F Pharmacy,2022-11-22 00:00:00.000000000,guest,147626,Enafix 100mg/5ml Suspension x1,Enafix 100mg/5ml Suspension x1,no data,no data,1,1.0,Enafix 100mg/5ml Suspension x1,Suspension,100MG/5ML,ENAFIX 100,Ernest Chemist,POM,Tier 2,Anti-Infectives,1,1.0,GREATER ACCRA
4,A & F Pharmacy,2022-11-22 00:00:00.000000000,guest,165608,DIPHEX ANTITUSSIVE x1,DIPHEX,DRUGS,Cough & Flu,1,1.0,DIPHEX ANTITUSSIVE x1,Syrup,7MG/5ML,DIPHEX,PHYTORIKER,OTC,Tier 4,Cough & Flu,1,1.0,GREATER ACCRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"882,275",Lifegem Pharmacy,2020-11-23 00:00:00.000000000,guest,165519,Cetapol PM 120mg/5ml Syrup x1,Cetapol PM 120mg/5ml Syrup x1,BABY AND CHILD HEALTH,Fever and Pain,1,1.0,Cetapol PM 120mg/5ml Syrup x1,Syrup,120MG/5ML,CETAPOL PM,PhytoRiker,OTC,Tier 4,Analgesics,1,1.0,ASHANTI
"882,276",Lifegem Pharmacy,2020-11-23 00:00:00.000000000,guest,165652,Foligrow 200ml Syrup x1,Foligrow 200ml Syrup,VITAMINS-AND-SUPPLEMENTS,Multivitamins,1,1.0,Foligrow 200ml Syrup x1,Syrup,1,FOLIGROW,Tobinco,OTC,Tier 3,Supplements,1,1.0,ASHANTI
"882,277",Lifegem Pharmacy,2020-11-23 00:00:00.000000000,guest,165709,Herbal Slimming Tea (21st Century) x1,Herbal Slimming Tea (21st Century) x1,no data,no data,1,1.0,Herbal Slimming Tea (21st Century) x1,Capsule,1,HERBAL SLIMMING TEA,21st Century,OTC,Tier 4,Supplements,1,1.0,ASHANTI
"882,278",Lifegem Pharmacy,2020-11-23 00:00:00.000000000,guest,169837,'Duplicate (Delete) Citro-C 100mg (10 Doses) Tab...,'Duplicate (Delete) Citro-C 100mg (10 Doses) Tab...,no data,no data,3,0.12,Citro-C 100mg (10 Doses) Tablets x25,Tablet,100MG,CITRO-C,A-EFAH,OTC,Tier 4,Supplements,25,0.12,ASHANTI


In [24]:
def col_rename(data):
    # Place all Column Names in a List
    col_names = data.get_column_names()
    # Renaming all columns to fit Vaex standards
    for i in col_names:
        new_name = i.replace(" ", "_").replace("-", "_")
        data.rename(i, new_name)
        col_names[col_names.index(i)] = new_name


col_rename(filtered_data)


In [25]:
df = filtered_data.drop(["Form", "Brand_/_Proprietary_Name", "Manufacturer", "OTC/POM", "Tier",
                        "Packs_Sold", "Vdl_Drug_Display_Name", "Customer_Type", "Item_Category", "Item_Sub_Category", "Strength", "Corresponding_Quantity", "Sum_of_Quantity_In_Units"])


In [26]:
date_max = df['Sale_Date'].values.max()
df_min = date_max - np.timedelta64(500, "D", )
df = df[df.Sale_Date >= df_min]


In [28]:
df = df.extract()
df.export_csv("%s\data\cleaned_data.csv"%path)


**Stock Balance Data Analysis**


In [29]:
# Importing Stock Balance Data
stk_balance = vx.read_csv('%s//data//stock_balance.csv' % path, dtype={
                          "Sum of stock balance in units": "float32", "Sum of stock_balance_in_packs": "float32",
                          'product id (vdl)': 'str'})

# Dropping Unwanted Columns
stk_balance.drop(columns=['inventory_date'], inplace=True)


#,facility_name,product id (vdl),product_name,Sum of stock balance in units,Sum of stock_balance_in_packs
0,CENTRIXMAS PHARMACY,106365,Ambesyl 10mg Tablets x30,30.0,1.0
1,CENTRIXMAS PHARMACY,106373,Atacand 32mg Tablets x28,56.0,2.0
2,CENTRIXMAS PHARMACY,106374,Atacand 8mg Tablets x28,56.0,2.0
3,CENTRIXMAS PHARMACY,106377,Atacand Plus 32mg/25mg Tablets x28,56.0,2.0
4,CENTRIXMAS PHARMACY,106378,Atenolol (Exeter) 50mg Tablets x28,28.0,1.0
...,...,...,...,...,...
"170,579",mPharma Delivery Pharmacy (GH),302899,Fiesta classic condom (3 pieces) x1,86.0,28.666666
"170,580",mPharma Delivery Pharmacy (GH),302901,Fiesta Dotted condom ( 3pieces) x1,58.0,19.333334
"170,581",mPharma Delivery Pharmacy (GH),302902,Fiesta Extra condom (3 pieces ) x1,36.0,12.0
"170,582",mPharma Delivery Pharmacy (GH),307294,Atenolol (Bristol) 50mg Tablet x28,868.0,31.0


In [30]:
# Filtering out only active facilities
stk_balance = stk_balance[stk_balance['facility_name'].isin(vfl_list)]


In [31]:
grouped_stk = stk_balance.groupby(by=['facility_name', 'product id (vdl)', 'product_name']).agg({
    'Sum of stock balance in units': "sum"})
grouped_stk = grouped_stk[grouped_stk['Sum of stock balance in units'] > 0]
grouped_stk.rename('product id (vdl)', 'Product ID')


'Product ID'

In [32]:
grouped_stk.join(vdl_vx, how='left', left_on='Product ID',
                 right_on='Product ID', rsuffix='_sc', inplace=True)
grouped_stk.drop("Product ID_sc", inplace=True)


#,facility_name,Product ID,product_name,Sum of stock balance in units,Product Description,Form,Strength,Brand / Proprietary Name,Manufacturer,OTC/POM,Tier,VDL Sub Category,Corresponding Quantity
0,Immaculate Pharmacy Ltd,109109,Clavulin 228mg/5ml Suspension x1,99.0,Clavulin 228mg/5ml Suspension x1,Suspension,228MG/5ML,CLAVULIN,GSK,POM,Tier 2,Anti-Infectives,1
1,Immaculate Pharmacy Ltd,186840,Pilex (100 Doses) Tablets x1,1708.0,Pilex (100 Doses) Tablets x1,Capsule,1,PILEX TABS,Himalaya,POM,Tier 3,herbals,100
2,Fresh Spring Chemists Ltd,108562,NOSPA 40MG/2ML X5,75.0,NOSPA 40MG/2ML X5,Ampoule,None,NOSPA,Sanofi,POM,Tier 1,Gastro-Intestinal,5
3,Fresh Spring Chemists Ltd,240955,Nurofen Cold & Flu Tablets x24,16.0,Nurofen Cold & Flu Tablets x24,Tablet,200mg/5mg,NUROFEN COLD & FLU TABS,Reckitt Benckiser Healthcare (UK) Ltd,OTC,Tier 2,Analgesics,1
4,Immaculate Pharmacy Ltd,266723,Symbicort 160mcg/4.5mcg (120 Doses) Turbuhaler x1,5.0,SYMBICORT 160/4.5 120 DOSES x1,Inhaler,160mcg/4.5mcg,SYMBICORT,AstraZeneca,POM,Tier 1,Anti-Asthma,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"51,861",Damykel Pharmacy,177162,Pregnacare Conception (30 Doses) Tablets x1,1.0,PREGNACARE CONCEPTION x1,Capsule,1,PREGNACARE,Vitabiotics,OTC,Tier 1,Supplements,1
"51,862",G & D Pharmacy,173259,Ekuro Bewu Cream x1,20.0,Ekuro Bewu Cream x1,Ointment,1,EKUROBEWU CREAM,Herbals,OTC,Tier 4,herbals,1
"51,863",Glomax Pharmacy,165676,FUNBACT-A TRIPLE ACTION x1,8.0,FUNBACT-A TRIPLE ACTION x1,Cream,None,FUNBACT-A,GV BLISS,OTC,Tier 3,Anti-Infectives,1
"51,864",Glomax Pharmacy,222743,"Zinc Oxide Plaster 1"" x1",8.0,"Zinc Oxide Plaster 1"" x1",Consumable,1,"ZINC OXIDE PLASTER 1""",Nantong Jianan Medical Products,OTC,Tier 4,Consumables,1


In [33]:
grouped_stk['Stock Balance'] = grouped_stk['Sum of stock balance in units'] / \
    grouped_stk['Corresponding Quantity']


In [36]:
grouped_stk = grouped_stk.extract()
grouped_stk.export_csv("%s//data//cleaned_stock_balance.csv"%path)


**Category Based Analysis**


In [ ]:
Loc_Cat_df = filtered_data.groupby(by=['VDL_Sub_Category'], agg={
    'Pack(s)_Sold': 'sum'}).sort(by='Pack(s)_Sold', ascending=False)


SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
location = filtered_data['LOCATION'].unique()


In [ ]:
px.line(data_frame=Loc_Cat_df, x='Sale_Date', y='Pack(s)_Sold')


In [ ]:
df = px.data.tips()
fig = px.scatter(df, x="total_bill", y="tip", color="sex", symbol="smoker", facet_col="time",
                 labels={"sex": "Gender", "smoker": "Smokes"})
fig.show()
